## **Dependencies**

In [128]:
import pandas as pd
from tqdm.notebook import tqdm

## **Load JSON**

Choose respective json file to transform to data frame
Json be found in /src/response_[ORG]

### Define Params

In [129]:
src_folder = "../../src/"
response_file = src_folder + "response_adf.json" # json format -> fetched in iati.ipynb
output_file = src_folder + "transformed_adf.csv"

In [130]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(10)

,sector_code,iati_identifier,title_narrative,document_link_url,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,activity_date_iso_date,recipient_country_code,location_name_narrative,reporting_org_narrative,title_narrative_xml_lang,description_narrative_xml_lang,recipient_region_code
0,[14022],46002-P-CI-EBA-005,"[ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO, Etud...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3]",5,[Cette étude vise à permettre au Gouvernement ...,2024-02-14T18:32:30Z,"[1997-06-11T00:00:00Z, 1998-02-11T00:00:00Z, 2...",[CI],"[Daloa, San Pédro]",[African Development Bank],"[en, fr]",[fr],NaN
1,[23111],46002-P-CI-F00-001,"[PRET D'AJUSTEMENT SECTORIEL ENERGIE, Programm...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[Le programme d'ajustement proposé du secteur ...,2024-02-14T18:32:31Z,"[1988-06-23T00:00:00Z, 1988-07-14T00:00:00Z, 1...",[CI],NaN,[African Development Bank],"[en, fr]",[fr],NaN
2,[23111],46002-P-CI-F00-002,"[INTERCONNECTION ELECTRIQUE, Interconnection E...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[Le projet d'interconnexion électrique a été a...,2024-02-14T18:32:31Z,"[1978-08-22T00:00:00Z, 1979-05-04T00:00:00Z, 1...",[CI],NaN,[African Development Bank],"[en, fr]",[fr],NaN
3,[22020],46002-P-CI-GB0-001,"[TELECOMMUNICATION PLATEAU III, Telecommunicat...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,"[Un pédagogue, un architecte et un ingénieur f...",2024-02-14T18:32:35Z,"[1982-03-16T00:00:00Z, 1982-06-28T00:00:00Z, 1...",[CI],NaN,[African Development Bank],"[en, fr]",[fr],NaN
4,[16050],46002-P-CI-I00-002,[PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[The present project concerns the post-crisis ...,2024-02-14T18:32:37Z,"[2007-12-05T00:00:00Z, 2008-02-28T00:00:00Z, 2...",[CI],"[Bafing Region, Denguele, Lacs, Montagnes, Moy...",[African Development Bank],"[en, fr]","[en, fr, en, fr, en, fr]",NaN
5,[14010],46002-P-ZM-E00-028,[Zambia - Transforming Rural Livelihoods in We...,[https://www.afdb.org/sites/all/libraries/pdf....,XM-DAC-46002,[1],"[1, 2, 3]",2,[The Transforming Rural Livelihoods in Western...,2024-02-14T07:46:34Z,"[2014-09-10T00:00:00Z, 2015-01-07T00:00:00Z, 2...",[ZM],"[Kalabo District, Kaoma District, Limulunga Di...",[African Development Bank],"[en, fr]","[en, fr, en, fr, en, fr]",NaN
6,[21023],46002-P-RW-DB0-010,[Rwanda - Gitarama-Ngororero-Mukamira Road Pro...,[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[The Gitarama-Ngororero-Mukamira Road Project ...,2024-02-14T06:37:04Z,"[2004-12-20T00:00:00Z, 2005-02-25T00:00:00Z, 2...",[RW],"[Muhanga District, Ngororero District, Nyabihu...",[African Development Bank],"[en, fr]","[en, fr, en, fr, en, fr]",NaN
7,[16050],46002-P-CI-I00-003,[Côte d’Ivoire - Social Inclusion and Cohesion...,[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[The Social Inclusion and Cohesion Enhancement...,2024-02-14T18:32:38Z,"[2014-06-13T00:00:00Z, 2014-06-25T00:00:00Z, 2...",[CI],"[Abidjan, Ivory Coast]",[African Development Bank],"[en, fr]","[en, fr, en, fr, en, fr]",NaN
8,[11110],46002-P-CI-IA0-001,"[Côte d'Ivoire - Education IV Project, Côte d'...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[The present project is the fourth phase of th...,2024-02-14T18:32:38Z,"[1992-10-01T00:00:00Z, 1992-10-16T00:00:00Z, 2...",[CI],"[Koumassi, Treichville, Bingerville, Kouassi-D...",[African Development Bank],"[en, fr]","[en, fr, en, fr, en, fr]",NaN
9,[11110],46002-P-CI-IA0-002,"[LYCEE PROFESSIONNEL PHASE I, Lycée Professio...",[https://projectsportal.afdb.org/dataportal/VP...,XM-DAC-46002,[1],"[1, 2, 3, 4]",3,[Dans la ligne des grandes orientations de la ...,2024-02

In [131]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 4427


XM-DAC-46002    4427
Name: reporting_org_ref, dtype: int64

In [132]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering****

### IATI ID

In [133]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,46002-P-CI-EBA-005
1,46002-P-CI-F00-001


### Title

In [134]:
################
# Add en title #
################

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # nan in pandas is type float
            # check if nan and if yes take first entry in lang
            if isinstance(lang_list, float):
                if isinstance(title_row, float):
                    trans_df["title_en"][index] = "NaN"
                else:
                    trans_df["title_en"][index] = title_row[0]
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if "en" or "EN" in lang_list:
                        if lang_list[j].lower() == "en":
                            title = title_row[j]
                            trans_df["title_en"][index] = title
                    else:
                        trans_df["title_en"][index] = "NaN"
            else:
                pass
        else:
            trans_df["title_en"][index] = row['title_narrative'][0]
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    pbar.update(1)

pbar.close()

trans_df.head(2)

  0%|          | 0/4427 [00:00<?, ?it/s]

,iati_id,title_en
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO"
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE


In [135]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # every title which is has no lang attribute is classified as English and therefore not in other
            if isinstance(lang_list, float):
                trans_df["title_other"][index] = "NaN"
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if lang_list[j].lower() != "en":
                        title = row['title_narrative'][j]
                        if trans_df["title_other"][index] == "NaN":
                            trans_df["title_other"][index] = title
                        else:
                            trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
            else:
                trans_df["title_other"][index] = title
            
            pbar.update(1)

        else:
            trans_df["title_other"][index] = "NaN"
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

pbar.close()

trans_df.head(200)

  0%|          | 0/4427 [00:00<?, ?it/s]

,iati_id,title_en,title_other
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa..."
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...
...,...,...,...
195,46002-P-GM-DB0-005,The Gambia - Trans-Gambia Corridor Preparatory...,Gambie - Projet des études préparatoires du co...
196,46002-P-UG-AB0-001,Uganda - Community Agricultural Infrastructure...,Ouganda - Programme d’amélioration de l’infras...
197,46002-P-TZ-BZ0-001,Industrial Rehabilitation and Trade Adjustment...,Ouganda - Programme d’amélioration de l’infras...
198,46002-P-GM-DD0-001,Banjul Port Project,Ouganda - Programme d’amélioration de l’infras...


### Organization

In [136]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

African Development Bank    4427
Name: organization, dtype: int64


,iati_id,title_en,title_other,organization
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank


### Country

In [137]:
trans_df["country"] = df["recipient_country_code"]
trans_df.head(5)

,iati_id,title_en,title_other,organization,country
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI]
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI]
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI]
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI]
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI]


### Region

In [138]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI],NaN
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI],NaN
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI],NaN


### Location

In [139]:
trans_df['location'] = df['location_name_narrative']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]"
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI],NaN,NaN
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI],NaN,NaN
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI],NaN,"[Bafing Region, Denguele, Lacs, Montagnes, Moy..."


### Descriptions

In [140]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        if 'description_narrative_xml_lang' in df.columns:
            descr_list = row['description_narrative_xml_lang']
            descr_row = row['description_narrative']

            # nan in pandas is type float
            # check if nan and if yes take first entry in descr
            if isinstance(descr_list, float):
                if isinstance(descr_row, float):
                    trans_df["description_en"][index] = "NaN"
                else:
                    trans_df["description_en"][index] = descr_row[0]
            else:
                # iterate throug description list
                for j in range(0, len(descr_list)):
                    # if description english
                    if descr_list[j].lower() == "en":
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_en"][index] == "NaN":
                            trans_df["description_en"][index] = descr
                        else:
                            trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                    else:
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_other"][index] == "NaN":
                            trans_df["description_other"][index] = descr
                        else:
                            trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
        else:
            descr_str = ""
            for d in row['description_narrative']:
                descr_str += f"{d}; "
            trans_df["description_en"][index] = descr_str

    except:
        print(f"Error: Index: {index} \n Row: {row}")
        
trans_df.head(5)

Error: Index: 14 
 Row: sector_code                                                                 [14010]
iati_identifier                                                  46002-P-ZM-EA0-001
title_narrative                   [Zambia - Kitwe Water Supply Study (ADF/TAF), ...
document_link_url                 [https://www.afdb.org/en/projects-and-operatio...
reporting_org_ref                                                      XM-DAC-46002
sector_vocabulary                                                               [1]
activity_date_type                                                     [1, 2, 3, 4]
activity_status_code                                                              3
description_narrative             [The objectives of the study are : i)  to iden...
last_updated_datetime                                          2024-02-14T07:46:37Z
activity_date_iso_date            [1989-11-22T00:00:00Z, 1989-12-06T00:00:00Z, 1...
recipient_country_code                              

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI],NaN,NaN,NaN,Le projet d'interconnexion électrique a été ap...
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI],NaN,NaN,NaN,"Un pédagogue, un architecte et un ingénieur fi..."
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI],NaN,"[Bafing Region, Denguele, Lacs, Montagnes, Moy...",The present project concerns the post-crisis m...,Le présent projet porte sur l'appui Institutio...


### Status

In [141]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...,Cancelled
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...,Finalisation


### Date

In [142]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...,Cancelled,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI],NaN,NaN,NaN,Le projet d'interconnexion électrique a été ap...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI],NaN,NaN,NaN,"Un pédagogue, un architecte et un ingénieur fi...",Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI],NaN,"[Bafing Region, Denguele, Lacs, Montagnes, Moy...",The present project concerns the post-crisis m...,Le présent projet porte sur l'appui Institutio...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,46002-P-NG-HAA-002,Nigeria - Domestic-oriented SME Financing Program,Nigeria - Programme de financement des PME loc...,African Development Bank,[NG],NaN,[Abuja],The intervention is for a Domestic-oriented SM...,La présente intervention concerne un programme...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
1796,46002-P-NG-HAA-004,Nigeria - Development Bank of Nigeria Plc Project,"Nigeria - Projet de ""Development Bank of Niger...",African Development Bank,[NG],NaN,[Abuja],The extension of long-term funding in a combin...,Le projet consiste à prolonger un financement ...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
1797,46002-P-CD-K00-002,Democratic Republic of Congo - Economic Recove...,République Démocratique du Congo - Programme d...,African Development Bank,[CD],NaN,[DR Congo],The Economic Reform and Reunification Support ...,Le Programme d’Appui à la Relance Economique e...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z
1798,46002-P-NG-IA0-001,Nigeria - Skills Training and Vocational Educa...,Nigéria - Projet de formation axée sur les com...,African Development Bank,[NG],NaN,"[Abia State, Kaduna State, Abuja, Ikare, Lassa...",The Nigeria Skills Training and Vocational Edu...,Le Projet de formation axée sur les compétence...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z


### Last Update

In [143]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...,Cancelled,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z,2024-02-14T18:32:30Z
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...,Finalisation,2024-01-17T00:00:00Z,2023-09-20T00:00:00Z,2026-12-31T00:00:00Z,2021-12-01T00:00:00Z,2024-02-14T18:32:31Z


### Sector Codes (CRS & SGDs)

In [144]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [145]:
crs5_df = pd.read_csv("../../src/crs5_codes.csv")
crs5_df.head(2)

,code,name,description,language,category,category-name,category-description
0,11110,Education policy and administrative management,"Education sector policy, planning and programm...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...
1,11120,Education facilities and training,"Educational buildings, equipment, materials; s...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...


In [146]:
crs3_df = pd.read_csv("../../src/crs3_codes.csv")
crs3_df.head(2)

,code,name,description,language,category,category-name,category-description
0,111,"Education, Level Unspecified",The codes in this category are to be used only...,en,NaN,NaN,NaN
1,112,Basic Education,NaN,en,NaN,NaN,NaN


In [147]:
sgd_goals_df = pd.read_csv("../../src/sgd_goals.csv")
sgd_goals_df.head(2)

,code,name,description,language,category,category-name,category-description
0,1,Goal 1. End poverty in all its forms everywhere,NaN,en,NaN,NaN,NaN
1,2,"Goal 2. End hunger, achieve food security and ...",NaN,en,NaN,NaN,NaN


In [148]:
sgd_targ_df = pd.read_csv("../../src/sgd_targets.csv")
sgd_targ_df.head(2)

,code,name,description,language,category,category-name,category-description
0,1.1,"By 2030, eradicate extreme poverty for all peo...",NaN,en,NaN,NaN,NaN
1,1.2,"By 2030, reduce at least by half the proportio...",NaN,en,NaN,NaN,NaN


In [149]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [150]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"

trans_df["sgd_goal_code"] = "NaN"
trans_df["sgd_goal_name"] = "NaN"

trans_df["sgd_target_code"] = "NaN"
trans_df["sgd_target_name"] = "NaN"

#trans_df["sgd_indicator_code"] = "NaN"
#trans_df["sgd_indicator_name"] = "NaN"

for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str

            # CRS 3
            crs3_str, crs3_codes = process_codes(combined_list, crs3_df, "2")
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

            # SGD Goals
            sgd_goal_str, sgd_goal_codes = process_codes(combined_list, sgd_goals_df, "7")
            trans_df["sgd_goal_code"][index] = sgd_goal_codes
            trans_df["sgd_goal_name"][index] = sgd_goal_str

            # SGD Target
            sgd_targ_str, sgd_targ_codes = process_codes(combined_list, sgd_targ_df, "8")
            trans_df["sgd_target_code"][index] = sgd_targ_codes
            trans_df["sgd_target_name"][index] = sgd_targ_str

            ## SGD Indicator
            #sgd_ind_str, sgd_ind_codes = process_codes(combined_list, sgd_ind_df, "9")
            #trans_df["sgd_indicator_code"][index] = sgd_ind_codes
            #trans_df["sgd_indicator_name"][index] = sgd_ind_str
        except:
            print(f"Error on Index {index}")
            pass

trans_df.head(100)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,sgd_goal_code,sgd_goal_name,sgd_target_code,sgd_target_name
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...,Cancelled,...,2021-12-01T00:00:00Z,2024-02-14T18:32:30Z,14022;,Sanitation - large systems;,NaN,NaN,NaN,NaN,NaN,NaN
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...,Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T18:32:31Z,23111;,"Energy sector policy, planning and administrat...",NaN,NaN,NaN,NaN,NaN,NaN
2,46002-P-CI-F00-002,INTERCONNECTION ELECTRIQUE,Interconnection Electrique,African Development Bank,[CI],NaN,NaN,NaN,Le projet d'interconnexion électrique a été ap...,Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T18:32:31Z,23111;,"Energy sector policy, planning and administrat...",NaN,NaN,NaN,NaN,NaN,NaN
3,46002-P-CI-GB0-001,TELECOMMUNICATION PLATEAU III,Telecommunication Plateau III,African Development Bank,[CI],NaN,NaN,NaN,"Un pédagogue, un architecte et un ingénieur fi...",Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T18:32:35Z,22020;,Telecommunications;,NaN,NaN,NaN,NaN,NaN,NaN
4,46002-P-CI-I00-002,PROJET D'APPUI INSTITUTIONNEL MULTISECTORIEL ...,Projet d'appui Institutionnel Multisectoriel ...,African Development Bank,[CI],NaN,"[Bafing Region, Denguele, Lacs, Montagnes, Moy...",The present project concerns the post-crisis m...,Le présent projet porte sur l'appui Institutio...,Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T18:32:37Z,16050;,Multisector aid for basic social services;,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,46002-P-ZM-KZ0-004,Zambia - Economic Recovery Loan II,Zambie - Deuxieme programme de redressement ec...,African Development Bank,[ZM],NaN,NaN,The second phase of Zambia Economic Recovery L...,La deuxième phase du prêt de redressement écon...,Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T07:48:09Z,51010;,General budget support-related aid;,NaN,NaN,NaN,NaN,NaN,NaN
96,46002-P-CI-FA0-024,Côte d'Ivoire - Project to Strengthen the Stru...,Côte d'Ivoire - Projet de renforcement des ouv...,African Development Bank,[CI],NaN,"[Vallée du Bandama, Comoé, Montagnes, Yamousso...",The present intervention relates to the first ...,La présente intervention est relative à la pre...,Implementation,...,2021-12-01T00:00:00Z,2024-02-14T18:32:34Z,23111;,"Energy sector policy, planning and administrat...",NaN,NaN,NaN,NaN,NaN,NaN
97,46002-P-ZM-AAF-002,Zambia - Aquaculture Enterprise Development Pr...,Zambie - Projet de promotion des entreprises a...,African Development Bank,[ZM],NaN,"[Siavonga, Mungwi District, Rufunsa District]",The Aquaculture Enterprise Development Project...,Le projet de développement des entreprises aqu...,Implementation,...,2021-12-01T00:00:00Z,2024-02-14T07:45:13Z,31320;,Fishery development;,NaN,NaN,NaN,NaN,NaN,NaN
98,46002-P-RW-IZ0-002,"Rwanda - Skills, Employability and Entrepreneu...","Rwanda - Programme pour les compétences, l’emp...",African Development Bank,[RW],NaN,"[Rwanda, Kigali]",The SEEP aims to protect critical employment c...,NaN,Finalisation,...,2021-12-01T00:00:00Z,2024-02-14T06:38:24Z,16050;,Multisector aid for basic social services;,NaN,NaN,NaN,NaN,NaN,NaN


### Documents

In [151]:
df.document_link_url.value_counts()

trans_df['docs'] = df['document_link_url']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,...,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,sgd_goal_code,sgd_goal_name,sgd_target_code,sgd_target_name,docs
0,46002-P-CI-EBA-005,"ETUDE ASSAIN. DALOA, GAGNOA & SAN-PEDRO","Etude d' Assainissement de Daloa, Gagnoa & Sa...",African Development Bank,[CI],NaN,"[Daloa, San Pédro]",NaN,Cette étude vise à permettre au Gouvernement d...,Cancelled,...,2024-02-14T18:32:30Z,14022;,Sanitation - large systems;,NaN,NaN,NaN,NaN,NaN,NaN,[https://projectsportal.afdb.org/dataportal/VP...
1,46002-P-CI-F00-001,PRET D'AJUSTEMENT SECTORIEL ENERGIE,Programme d'ajustement du secteur de l'énergie,African Development Bank,[CI],NaN,NaN,NaN,Le programme d'ajustement proposé du secteur d...,Finalisation,...,2024-02-14T18:32:31Z,23111;,"Energy sector policy, planning and administrat...",NaN,NaN,NaN,NaN,NaN,NaN,[https://projectsportal.afdb.org/dataportal/VP...


## **Save DF as CSV**

In [152]:
trans_df.to_csv(output_file, index=False) 